In [ ]:
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from decimal import Decimal
from benchmark.alphaxutils import *

import requests
import pandas as pd

In [ ]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = '18'
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = str(cgtokens.decimals[i])

In [ ]:
threshold_orders = 6
support_partial = True

# # ETH side orders
# tkn_y = 'ETH'
# tkn_x = 'USDC'

# realisticByTarget_inputAmount = 10 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
# realisticBySource_inputAmount = 30000 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

# order_params = {
#     'liquidity':'5',
#     'highestRate':'0.0005', # 2000
#     'lowestRate':'0.0004',  # 2500
#     'marginalRate':'0.0005',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params2 = {
#     'liquidity':'4',
#     'highestRate':'0.0006', # 1666
#     'lowestRate':'0.0005',  # 2000
#     'marginalRate':'0.0006',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params3 = {
#     'liquidity':Decimal('10'),
#     'highestRate':Decimal('0.0006'), # 1666
#     'lowestRate':Decimal('0.0005'),  # 2000
#     'marginalRate':Decimal('0.0006'),
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }


# USDC side orders
tkn_y = 'USDC'
tkn_x = 'ETH'

realisticByTarget_inputAmount = 30000 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
realisticBySource_inputAmount = 10 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

order_params = {
    'liquidity':'20000',
    'highestRate':'2500',
    'lowestRate':'2000',
    'marginalRate':'2500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params2 = {
    'liquidity':'17000',
    'highestRate':'3500',
    'lowestRate':'3000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params3 = {
    'liquidity':'15000',
    'highestRate':'3500',
    'lowestRate':'2000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }

orders = []
for i in range(1):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]
    orders += [Order(order_params3)]

In [ ]:
print("inputAmount",realisticByTarget_inputAmount)
TargetActions2 = mpr_matchByTarget(realisticByTarget_inputAmount, orders, threshold_orders, support_partial)
total_output_Target2 = sum([TargetActions2[k]['dx'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_input_Target2 = sum([TargetActions2[k]['dy_specified'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Target2)
print("Total Return Real:", total_output_Target2)
print("Rate Real:",(total_input_Target2/total_output_Target2))
TargetActions2

In [ ]:
print("inputAmount",realisticBySource_inputAmount)
SourceActions2 = mpr_matchBySource(realisticBySource_inputAmount, orders, threshold_orders, support_partial)
total_input_Source2 = sum([SourceActions2[k]['dx_specified'] for k in SourceActions2.keys()])  / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_output_Source2 = sum([SourceActions2[k]['dy'] for k in SourceActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Source2)
print("Total Return:", total_output_Source2)
print("Rate:",total_output_Source2/total_input_Source2)
SourceActions2

In [ ]:
from benchmark import util
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from benchmark.alphaxutils import Order

test = {
    "amount": "15000000",
    "orders": [
        {
            "liquidity": "736740",
            "lowestRate": "0.250000000000",
            "highestRate": "0.500000000000",
            "marginalRate": "0.498419625806"
        },
        {
            "liquidity": "981654",
            "lowestRate": "0.500000000000",
            "highestRate": "0.750000000000",
            "marginalRate": "0.748328756607"
        },
        {
            "liquidity": "1228567",
            "lowestRate": "0.750000000000",
            "highestRate": "1",
            "marginalRate": "0.998698189928"
        },
        {
            "liquidity": "1477481",
            "lowestRate": "1",
            "highestRate": "1.250000000000",
            "marginalRate": "1.249287484872"
        },
        {
            "liquidity": "1728394",
            "lowestRate": "1.250000000000",
            "highestRate": "1.500000000000",
            "marginalRate": "1.500000000000"
        },
        {
            "liquidity": "1975308",
            "lowestRate": "1.500000000000",
            "highestRate": "1.750000000000",
            "marginalRate": "1.750000000000"
        },
        {
            "liquidity": "2222221",
            "lowestRate": "1.750000000000",
            "highestRate": "2",
            "marginalRate": "2"
        },
        {
            "liquidity": "2469135",
            "lowestRate": "2",
            "highestRate": "2.250000000000",
            "marginalRate": "2.250000000000"
        },
        {
            "liquidity": "2716048",
            "lowestRate": "2.250000000000",
            "highestRate": "2.500000000000",
            "marginalRate": "2.500000000000"
        },
        {
            "liquidity": "2962962",
            "lowestRate": "2.500000000000",
            "highestRate": "2.750000000000",
            "marginalRate": "2.750000000000"
        }
    ]
}

oldMethodOutputActions = [
    {
        "id": "9",
        "input": "2962962",
        "output": "1130030"
    },
    {
        "id": "8",
        "input": "2716048",
        "output": "1145187"
    },
    {
        "id": "7",
        "input": "2469135",
        "output": "1163962"
    },
    {
        "id": "6",
        "input": "2222221",
        "output": "1187828"
    },
    {
        "id": "5",
        "input": "1975308",
        "output": "1219187"
    },
    {
        "id": "4",
        "input": "1728394",
        "output": "1262241"
    },
    {
        "id": "3",
        "input": "925932",
        "output": "793544"
    }
]

method = mpr_matchByTarget

amount = int(test['amount'])

orders = [Order({**order_params, 'dec_y': 0, 'dec_x': 0}) for order_params in test['orders']]

newMethodOutputActions = mpr_matchByTarget(amount, orders, len(oldMethodOutputActions) + 3, True)

oldInputSum  = sum(int(action['input'       ]) for action in oldMethodOutputActions)
newInputSum  = sum(int(action['dy_specified']) for action in newMethodOutputActions.values())
oldOutputSum = sum(int(action['output'      ]) for action in oldMethodOutputActions)
newOutputSum = sum(int(action['dx'          ]) for action in newMethodOutputActions.values())

newMethodOutputActions = [
    {
        'id': key,
        'input': str(newMethodOutputActions[key]['dy_specified']),
        'output': str(newMethodOutputActions[key]['dx'])
    }
    for key in newMethodOutputActions
]

print('oldActions =', util.dumps(oldMethodOutputActions, indent = 4))
print('newActions =', util.dumps(newMethodOutputActions, indent = 4))
print('oldInputSum  =', oldInputSum )
print('newInputSum  =', newInputSum )
print('oldOutputSum =', oldOutputSum)
print('newOutputSum =', newOutputSum)



In [1]:
from benchmark import util
from benchmark.core.trade import impl
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from benchmark.alphaxutils import Order

MAX_NUM_OF_ADDITIONAL_ORDERS = 0

for fileName in ['ArbitraryMatch', 'BigPoolMatch', 'EthUsdcMatch', 'SpecialMatch']:
    for test in util.read('', f'benchmark/resources/match/{fileName}'):
        if test['method'] == 'matchBySourceAmount':
            method = mpr_matchBySource
            attr1 = 'dx_specified'
            attr2 = 'dy'
            direction = +1
        if test['method'] == 'matchByTargetAmount':
            method = mpr_matchByTarget
            attr1 = 'dy_specified'
            attr2 = 'dx'
            direction = -1
        amount = int(test['amount'])
        orders = [Order({**order_params, 'dec_y': 0, 'dec_x': 0}) for order_params in test['orders']]
        oldActions = test['actions']
        for numOfAdditionalOrders in range(MAX_NUM_OF_ADDITIONAL_ORDERS + 1):
            threshold    = len(oldActions) + numOfAdditionalOrders
            newActions   = method(amount, orders, threshold, True)
            oldInputSum  = sum(int(action['input' ]) for action in oldActions)
            newInputSum  = sum(int(action[attr1   ]) for action in newActions.values())
            oldOutputSum = sum(int(action['output']) for action in oldActions)
            newOutputSum = sum(int(action[attr2   ]) for action in newActions.values())
            # for key in newActions.keys():
            #     action = newActions[key]
            #     output = impl.execute(test['method'].replace('match', 'trade'), action[attr1], test['orders'][key])
            #     assert output == action[attr2], f'{output} {action[attr2]}'
            OK = newOutputSum * direction >= oldOutputSum * direction
            if OK:
                break
        if not OK:
            print(util.dumps(test, indent = 4))
            for key in newActions:
                obj = {
                    'id': key,
                    'input': str(newActions[key][attr1]),
                    'output': str(newActions[key][attr2]),
                }
                print(util.dumps(obj, indent = 4))
            print('oldInputSum  =', oldInputSum )
            print('newInputSum  =', newInputSum )
            print('oldOutputSum =', oldOutputSum)
            print('newOutputSum =', newOutputSum)
            assert False


AssertionError: 10544553 10544554